# **Bitcoin price forecasting - Random Forest**
### Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author
Corsi Danilo - corsi.1742375@studenti.uniroma1.it



# Dependencies, Libraries and Tools

In [1]:
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
SLOW_OPERATION = False

In [2]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
#Install some useful dependencies
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from itertools import cycle

import plotly.express as px

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import gc

import pandas as pd
import numpy as np
import json

import matplotlib.pyplot as plt
import seaborn as sns

# !pip install -U -q PyDrive # To use files that are stored in Google Drive directly (e.g., without downloading them from an external URL)
# !apt install openjdk-8-jdk-headless -qq
# import os
# os.environ["JAVA_HOME"] = JAVA_HOME

In [4]:
# Install Spark and related dependencies
!pip install pyspark

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
from pyspark.ml import Pipeline

# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '4G').\
                set('spark.driver.memory', '45G').\
                set('spark.driver.maxResultSize', '10G').\
                set("spark.kryoserializer.buffer.max", "1G").\
                setAppName("BitcoinPriceForecasting").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=ae215309f327fcf4a58c0afb9ffc7db3ddc3ed9d8b03780e85d45657d162d0ed
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


# Link to Google Drive

In [5]:
# Define GDrive paths
GDRIVE_DIR = "/content/drive"

GDRIVE_DATASET_OUTPUT_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/datasets/output"

GDRIVE_DATASET_NAME = "bitcoin_blockchain_data_30min"
GDRIVE_DATASET_NAME_TEST = GDRIVE_DATASET_NAME + "_test"

GDRIVE_DATASET_NAME_EXT_TEST = "/" + GDRIVE_DATASET_NAME_TEST + ".parquet"

GDRIVE_DATASET_TEST = GDRIVE_DATASET_OUTPUT_DIR + GDRIVE_DATASET_NAME_EXT_TEST

In [6]:
# Point Colaboratory to our Google Drive
from google.colab import drive

drive.mount(GDRIVE_DIR, force_remount=True)

Mounted at /content/drive


# Testing the models ❗

In [7]:
# Load datasets into pyspark dataframe objects
test_df = spark.read.load(GDRIVE_DATASET_TEST,
                         format="parquet",
                         sep=",",
                         inferSchema="true",
                         header="true"
                    )

In [8]:
PATH_MODELS = "/content/drive/MyDrive/BDC/project/models/"

LR_MODEL_NAME = "LinearRegression"
GLR_MODEL_NAME = "GeneralizedLinearRegression"
RF_MODEL_NAME = "RandomForestRegressor"
GBT_MODEL_NAME = "GBTRegressor"

In [9]:
GDRIVE_FEATURES_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/features"

GDRIVE_ALL_FEATURES_NAME = "all_features"
GDRIVE_MORE_REL_FEATURES_NAME = "more_rel_features"
GDRIVE_LESS_REL_FEATURES_NAME = "less_rel_features"

GDRIVE_ALL_FEATURES_NAME_EXT = "/" + GDRIVE_ALL_FEATURES_NAME + ".json"
GDRIVE_MORE_REL_FEATURES_NAME_EXT = "/" + GDRIVE_MORE_REL_FEATURES_NAME + ".json"
GDRIVE_LESS_REL_FEATURES_NAME_EXT = "/" + GDRIVE_LESS_REL_FEATURES_NAME + ".json"

GDRIVE_ALL_FEATURES = GDRIVE_FEATURES_DIR + GDRIVE_ALL_FEATURES_NAME_EXT
GDRIVE_MORE_REL_FEATURES = GDRIVE_FEATURES_DIR + GDRIVE_MORE_REL_FEATURES_NAME_EXT
GDRIVE_LESS_REL_FEATURES = GDRIVE_FEATURES_DIR + GDRIVE_LESS_REL_FEATURES_NAME_EXT

In [10]:
# Set the depended variable
TARGET_VAL = 'market-price'

# Set the features label
FEATURES_LABEL = "features"

In [11]:
# # Loading correlation matrix features
# with open(GDRIVE_ALL_FEATURES, "r") as f:
#     all_features = json.load(f)
# print(all_features)

In [12]:
# Loading correlation matrix features
with open(GDRIVE_MORE_REL_FEATURES, "r") as f:
    more_rel_features = json.load(f)
print(more_rel_features)

['market-cap', 'miners-revenue', 'sma-5-days', 'sma-7-days', 'estimated-transaction-volume-usd', 'sma-10-days', 'n-transactions-total', 'blocks-size', 'sma-100-days', 'total-bitcoins']


In [13]:
# # Loading correlation matrix features
# with open(GDRIVE_LESS_REL_FEATURES, "r") as f:
#     less_rel_features = json.load(f)
# print(less_rel_features)

In [14]:
# Upload the 3 templates saved in Google Drive
from pyspark.ml import PipelineModel

lr = PipelineModel.load(PATH_MODELS + LR_MODEL_NAME)
glr = PipelineModel.load(PATH_MODELS + GLR_MODEL_NAME)
rf = PipelineModel.load(PATH_MODELS + RF_MODEL_NAME)
gbt = PipelineModel.load(PATH_MODELS + GBT_MODEL_NAME)

In [15]:
def select_features(dataset, features, featureCol, labelCol):
  vectorAssembler = VectorAssembler(
    inputCols = features,
    outputCol = featureCol)

  dataset = vectorAssembler.transform(dataset)
  dataset = dataset.select(['timestamp','id', featureCol, labelCol])

  return dataset

In [16]:
from sklearn.metrics import mean_absolute_percentage_error

def test_final_model(dataframe, trained_model, features, features_label, target_val, ml_model):
  dataframe = select_features(dataframe, features, features_label, target_val)

  # Make predictions
  predictions = trained_model.transform(dataframe)

  results = []

  # Compute validation error by several evaluator
  rmse_evaluator = RegressionEvaluator(labelCol=target_val, predictionCol="prediction", metricName='rmse')
  mae_evaluator = RegressionEvaluator(labelCol=target_val, predictionCol="prediction", metricName='mae')
  r2_evaluator = RegressionEvaluator(labelCol=target_val, predictionCol="prediction", metricName='r2')
  var_evaluator = RegressionEvaluator(labelCol=target_val, predictionCol="prediction", metricName='var')

  predictions_pd = predictions.select(target_val, "prediction").toPandas()
  mape = mean_absolute_percentage_error(predictions_pd[target_val], predictions_pd["prediction"])

  rmse = rmse_evaluator.evaluate(predictions)
  mae = mae_evaluator.evaluate(predictions)
  var = var_evaluator.evaluate(predictions)
  r2 = r2_evaluator.evaluate(predictions)
  # Adjusted R-squared
  n = predictions.count()
  p = len(predictions.columns)
  adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)

  # Use dict to store each result
  result = {
      "Model": ml_model,
      "Type": "trained",
      "RMSE": rmse,
      "MAPE":mape,
      "MAE": mae,
      "Variance": var,
      "R2": r2,
      "Adjusted_R2": adj_r2,
  }

  # Transform dict to pandas dataframe
  result_df = pd.DataFrame(result, index=[0])

  return result_df, predictions.toPandas()

In [17]:
lr_results, lr_predictions = test_final_model(test_df, lr, more_rel_features, FEATURES_LABEL, TARGET_VAL, LR_MODEL_NAME)
glr_results, glr_predictions = test_final_model(test_df, glr, more_rel_features, FEATURES_LABEL, TARGET_VAL, GLR_MODEL_NAME)
rf_results, rf_predictions = test_final_model(test_df, rf, more_rel_features, FEATURES_LABEL, TARGET_VAL, RF_MODEL_NAME)
gbt_results, gbt_predictions = test_final_model(test_df, gbt, more_rel_features, FEATURES_LABEL, TARGET_VAL, GBT_MODEL_NAME)

In [18]:
# Define what model_info and evaluators in the Model Comparison Table
model_info = ['Model','Type']
evaluator_lst = ['RMSE','MAPE','MAE','Variance','R2','Adjusted_R2']

# The the Cross Validation results would like to compare
comparison_lst = [lr_results, glr_results, rf_results, gbt_results]

In [19]:
def modelComparison(cv_result, model_info, evaluator_lst):
    # Calculate mean of all splits on chosen evaluator
    col_mean_df = cv_result[evaluator_lst].mean().to_frame().T
    # Extract model info
    model_info_df = cv_result[model_info][:1]
    # Concatenate by row
    comparison_df = pd.concat([model_info_df,col_mean_df],axis=1)
    return comparison_df

In [20]:
# Show the Comparison Table
pd.concat([modelComparison(cv_result, model_info, evaluator_lst) for cv_result in comparison_lst])

,Model,Type,RMSE,MAPE,MAE,Variance,R2,Adjusted_R2
0,LinearRegression,trained,414.443168,0.011568,344.568550,483283.745029,0.446146,0.444915
0,GeneralizedLinearRegression,trained,267.564622,0.006820,203.536685,423094.819492,0.769154,0.768641
0,RandomForestRegressor,trained,638.275769,0.017433,515.842349,519498.479379,-0.313658,-0.316578
0,GBTRegressor,trained,658.625463,0.018902,558.814938,457376.183246,-0.398759,-0.401867


In [21]:
def show_results(test_df, lr_predictions, glr_predictions, rf_predictions, gbt_predictions):
  trace1 = go.Scatter(
      x = test_df['timestamp'],
      y = test_df['market-price'].astype(float),
      mode = 'lines',
      name = 'Test set'
  )

  trace2 = go.Scatter(
      x = lr_predictions['timestamp'],
      y = lr_predictions['prediction'].astype(float),
      mode = 'lines',
      name = 'Linear Regression predictions'
  )

  trace2 = go.Scatter(
      x = lr_predictions['timestamp'],
      y = lr_predictions['prediction'].astype(float),
      mode = 'lines',
      name = 'Linear Regression predictions'
  )

  trace3 = go.Scatter(
      x = glr_predictions['timestamp'],
      y = glr_predictions['prediction'].astype(float),
      mode = 'lines',
      name = 'Generalized Linear Regression predictions'
  )

  trace4 = go.Scatter(
      x = rf_predictions['timestamp'],
      y = rf_predictions['prediction'].astype(float),
      mode = 'lines',
      name = 'Random Forest Regressor predictions'
  )

  trace5 = go.Scatter(
      x = gbt_predictions['timestamp'],
      y = gbt_predictions['prediction'].astype(float),
      mode = 'lines',
      name = 'GBTRegressor predictions'
  )

  layout = dict(
      title='Test and predictions with Rangeslider',
      xaxis=dict(
          rangeselector=dict(
              buttons=list([
                  #change the count to desired amount of months.
                  dict(count=1,
                      label='1m',
                      step='month',
                      stepmode='backward'),
                  dict(count=6,
                      label='6m',
                      step='month',
                      stepmode='backward'),
                  dict(count=12,
                      label='1y',
                      step='month',
                      stepmode='backward'),
                  dict(count=36,
                      label='3y',
                      step='month',
                      stepmode='backward'),
                  dict(step='all')
              ])
          ),
          rangeslider=dict(
              visible = True
          ),
          type='date'
      )
  )

  data = [trace1,trace2,trace3, trace4, trace5]
  fig = dict(data=data, layout=layout)
  iplot(fig, filename = "Test and predictions with Rangeslider")

In [22]:
show_results(test_df.toPandas(), lr_predictions, glr_predictions, rf_predictions, gbt_predictions)